In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

from itertools import combinations
from collections import Counter

In [44]:
df = pd.read_csv('pokemon_data.csv')

In [45]:
df.columns

Index(['ID', 'Name', 'HP', 'Attack', 'Defense', 'Sp. Attack', 'Sp. Defense',
       'Speed', 'Base_Stats', 'normal_weakness', 'fire_weakness',
       'water_weakness', 'electric_weakness', 'grass_weakness', 'ice_weakness',
       'fighting_weakness', 'poison_weakness', 'ground_weakness',
       'flying_weakness', 'psychic_weakness', 'bug_weakness', 'rock_weakness',
       'ghost_weakness', 'dragon_weakness', 'dark_weakness', 'steel_weakness',
       'fairy_weakness', 'height_inches', 'height_meters', 'weight_pounds',
       'weight_kilograms', 'capturing_rate', 'gender_male_ratio', 'egg_steps',
       'egg_cycles', 'abilities', 'Type 1', 'Type 2', 'Classification_info',
       'Forms', 'gen', 'Is_Legendary', 'Is_Mythical', 'Is_Ultra_Beast',
       'number_immune', 'number_not_effective', 'number_normal',
       'number_super_effective'],
      dtype='object')

# Tipos de Pokémon

In [46]:
pokemon_by_type = df.groupby(['Type 1']).count()

# Tipo 1 é o tipo principal do pokémon
# Tipo 2 é o tipo secundário do pokémon

In [47]:
fig_pokemon_by_type = px.bar(pokemon_by_type, x='ID')

In [48]:
# essa figura agrupa os pokemon por seus tipos principais
fig_pokemon_by_type

## Relação entre tipos

In [49]:
pokemon_by_type_type = df.groupby(['Type 1', 'Type 2']).size().reset_index(name='contagem')

In [50]:
# aqui vou criar um visual para filtrar tipos de pokémon, vou buscar entender quais os relacionamentos mais comuns entre tipos.
# utilize esse visual, caso queira ver o gráfico com mais detalhes
fig_pokemon_by_type_type = px.sunburst(pokemon_by_type_type, path=['Type 1','Type 2'], values='contagem', title='Relação entre tipos de pokémon', width=3840, height=2160)

In [51]:
# utilize esse visual para a imagem aberta.
fig_pokemon_by_type_type = px.sunburst(pokemon_by_type_type, path=['Type 1','Type 2'], values='contagem', title='Relação entre tipos de pokémon')

In [52]:
fig_pokemon_by_type_type

In [53]:
legends_per_type = px.box(
    df, 
    x="Is_Legendary", 
    y="Base_Stats", 
    color="Type 1",
    title="Estatísticas Base por Status Lendário"
)


In [54]:
legends_per_type

## Combate

In [55]:
# Visualizando a relação peso x defesa, 
weight_defense = px.scatter(
    df, 
    x="weight_kilograms", 
    y="Defense", 
    color="Type 1",
    trendline="lowess", # ative essa linha caso queira ver a trendline
    title="Peso vs. Defesa")

## Relação Peso x Defesa

In [56]:
weight_defense

# Visualizando a imagem, além da relação com peso, podemos ver uma relação com tipos, Rock, Ground, Steel e Bug como destaque

In [57]:
# Visualizando a relação peso x ataque, 
weight_attack = px.scatter(
    df, 
    x="weight_kilograms", 
    y="Attack", 
    color="Type 1",
    trendline="lowess", # ative essa linha caso queira ver a trendline
    title="Peso vs. Ataque")

## Relação Peso x Ataque

In [58]:
weight_attack

# Normal e dragon são os que se encaixam melhor na relação.

In [59]:
weakness_counts = (df[[col for col in df.columns if '_weakness' in col]] >= 2).sum().sort_values()

# Criar gráfico
fig1 = px.bar(
    x=weakness_counts.values,
    y=[col.replace('_weakness', '') for col in weakness_counts.index],
    orientation='h',
    title='Pokémons com Fraqueza (Multiplicador ≥ 2x)',
    labels={'x': 'Número de Pokémons', 'y': 'Tipo de Ataque'},
    color=weakness_counts.values,
    color_continuous_scale='reds'
)

fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()

In [60]:
# Agrupar por Type 1 e calcular % de fraquezas dos pokemon
type_weakness_pct = (
    df.groupby('Type 1')[[col for col in df.columns if '_weakness' in col]]
    .apply(lambda x: x.mean())  # % de Pokémons com fraqueza
)

# Heatmap
type_weakness_fig = px.imshow(
    type_weakness_pct,
    title='Relação entre tipos e fraquezas',
    labels=dict(x="Tipo de Ataque", y="Tipo Primário", color="% com Fraqueza"),
    color_continuous_scale='viridis',
    aspect="auto"
)

type_weakness_fig.update_xaxes(side="top")
type_weakness_fig.show()

### Sofrem para atacar.

Aqui vamos ver os tipos que sofrem com a maior recorrência de resistência.
Isto é, contaremos quantos pokémon são resistentes a seus ataques.

In [61]:
# Contar resistências (valores <= 0.5)
resistance_counts = (df[[col for col in df.columns if '_weakness' in col]] <= 0.5).sum().sort_values()

resistance_fig = px.bar(
    x=resistance_counts.values,
    y=[col.replace('_weakness', '') for col in resistance_counts.index],
    orientation='h',
    title='Pokémons com Resistência (Multiplicador ≤ 0.5x)',
    labels={'x': 'Número de Pokémons', 'y': 'Tipo de Ataque'},
    color=resistance_counts.values,
    color_continuous_scale='blues'
)

resistance_fig.update_layout(yaxis={'categoryorder':'total ascending'})
resistance_fig.show()